In [1]:
## Calulate ATR need high, low, and previous close

In [41]:
import mysql.connector
import pandas as pd
import numpy as np

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = "select * from (SELECT effective_date, `adj_high`, `adj_low`, `adj_close` FROM `market.stock_price` \
            WHERE symbol = '{0}' and effective_date >= '2017-01-01' order \
            by effective_date DESC limit 1000) sub order by effective_date ASC".format(symbol)
    
    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    return dfsql

In [53]:
df = load_stockdata('AAPL')
df.head()

,adj_high,adj_low,adj_close
effective_date,,,
2017-01-03,114.893155,113.342546,114.715378
2017-01-04,115.070931,114.320318,114.586983
2017-01-05,115.420756,114.379577,115.169696
2017-01-06,116.700551,115.031425,116.453639
2017-01-09,117.954865,116.483269,117.520300


In [54]:
df['h_l'] = df['adj_high'] - df ['adj_low']
df['h_pc'] = abs(df['adj_high'] - df ['adj_close'].shift(1))
df['l_pc'] = abs(df['adj_low'] - df ['adj_close'].shift(1))
df['TR'] = df[['h_l','h_pc','l_pc']].max(axis=1)
df.head(30)

,adj_high,adj_low,adj_close,h_l,h_pc,l_pc,TR
effective_date,,,,,,,
2017-01-03,114.893155,113.342546,114.715378,1.550609,NaN,NaN,1.550609
2017-01-04,115.070931,114.320318,114.586983,0.750613,0.355553,0.395060,0.750613
2017-01-05,115.420756,114.379577,115.169696,1.041179,0.833773,0.207406,1.041179
2017-01-06,116.700551,115.031425,116.453639,1.669126,1.530855,0.138271,1.669126
2017-01-09,117.954865,116.483269,117.520300,1.471596,1.501226,0.029630,1.501226
2017-01-10,117.905483,116.838822,117.638817,1.066661,0.385183,0.681478,1.066661
2017-01-11,118.448689,117.135117,118.270913,1.313572,0.809872,0.503700,1.313572
2017-01-12,117.826471,116.749934,117.777088,1.076537,0.444442,1.520979,1.520979
2017-01-13,118.142518,117.342523,117.569682,0.799995,0.365430,0.434565,0.799995


## 10 dsys ATR by
- get SMA for the first 10 days
- then apply formula [(Prior ATR * (period-1)) + Current TR] / period

In [68]:
# get the first SMA for the rolling windows
period = 14
rolling = df[:period]["TR"].rolling(period)
#rolling.mean()

# combine the first SMA with the rest close adj price

tr_con = pd.concat([rolling.mean(), df[period:]['TR']])
df['tr_con'] = tr_con

for i in range(1, len(tr_con)):
    if (not pd.isnull(tr_con[i-1])):
        tr_con[i] = ( tr_con[i-1] * (period-1) + tr_con[i] ) / period

# tr_con has been trasformed to ATR
df['atr'] = tr_con
df['ema'] = df['tr_con'].ewm(span=period, adjust=False).mean()

df.head(30)

#dfewma = dfewma.where((pd.notnull(dfewma)),None)
#dfewma.head(30)

,adj_high,adj_low,adj_close,h_l,h_pc,l_pc,TR,tr_con,atr,ema
effective_date,,,,,,,,,,
2017-01-03,114.893155,113.342546,114.715378,1.550609,NaN,NaN,1.550609,NaN,NaN,NaN
2017-01-04,115.070931,114.320318,114.586983,0.750613,0.355553,0.395060,0.750613,NaN,NaN,NaN
2017-01-05,115.420756,114.379577,115.169696,1.041179,0.833773,0.207406,1.041179,NaN,NaN,NaN
2017-01-06,116.700551,115.031425,116.453639,1.669126,1.530855,0.138271,1.669126,NaN,NaN,NaN
2017-01-09,117.954865,116.483269,117.520300,1.471596,1.501226,0.029630,1.501226,NaN,NaN,NaN
2017-01-10,117.905483,116.838822,117.638817,1.066661,0.385183,0.681478,1.066661,NaN,NaN,NaN
2017-01-11,118.448689,117.135117,118.270913,1.313572,0.809872,0.503700,1.313572,NaN,NaN,NaN
2017-01-12,117.826471,116.749934,117.777088,1.076537,0.444442,1.520979,1.520979,NaN,NaN,NaN
2017-01-13,118.142518,117.342523,117.569682,0.799995,0.365430,0.434565,0.799995,NaN,NaN,NaN


## function for Keltner Chennals

In [38]:
def process_keltnerchannels(df):
    
    # ==> 20 days EWMA
    ema_period = 20
    ema_rolling = df[:ema_period]["adj_close"].rolling(ema_period)

    # combine the first SMA with the rest close adj price
    ema_con = pd.concat([ema_rolling.mean(), df[ema_period:]['adj_close']])
    #con.head(30)

    df['ema_con'] = ema_con
    df['ema20'] = ema_con.ewm(span=ema_period, adjust=False).mean()
    
    # ==> 10 days ATR
    atr_period = 10

    # prepare TR for each day
    df['h_l'] = df['adj_high'] - df ['adj_low']
    df['h_pc'] = abs(df['adj_high'] - df ['adj_close'].shift(1))
    df['l_pc'] = abs(df['adj_low'] - df ['adj_close'].shift(1))
    df['TR'] = df[['h_l','h_pc','l_pc']].max(axis=1)

    # get the first SMA for the rolling windows
    atr_rolling = df[:atr_period]["TR"].rolling(atr_period)

    # combine the first SMA with the rest close adj price
    tr_con = pd.concat([atr_rolling.mean(), df[atr_period:]['TR']])
    df['tr_con'] = tr_con
    atr_cal = (df['tr_con'].shift(1) * (atr_period-1) + df['tr_con']) / atr_period
    df['atr_cal'] = atr_cal
    # merge the first SMA with the rest ATR
    df['atr10'] = pd.concat([tr_con[:atr_period], atr_cal[atr_period:]])


    # ==> assign upper/lower band
    df['atr1_upper'] = df['ema20'] + df['atr10'] 
    df['atr1_lower'] = df['ema20'] - df['atr10']
    df['atr2_upper'] = df['ema20'] + df['atr10'] * 2
    df['atr2_lower'] = df['ema20'] - df['atr10'] * 2
    df['atr3_upper'] = df['ema20'] + df['atr10'] * 3
    df['atr3_lower'] = df['ema20'] - df['atr10'] * 3

    return df;


In [40]:
df_kc = process_keltnerchannels(df)
df_kc.head(30)

,adj_high,adj_low,adj_close,ema_con,ema20,h_l,h_pc,l_pc,TR,tr_con,atr_cal,atr10,atr1_upper,atr1_lower,atr2_upper,atr2_lower,atr3_upper,atr3_lower
effective_date,,,,,,,,,,,,,,,,,,
2014-02-27,34.447387,33.847353,34.420112,NaN,NaN,0.600034,NaN,NaN,0.600034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-02-28,34.965597,34.383747,34.829226,NaN,NaN,0.581850,0.545485,0.036365,0.581850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-03,34.665581,34.088276,34.347381,NaN,NaN,0.577305,0.163645,0.740950,0.740950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-04,34.983780,34.611032,34.920140,NaN,NaN,0.372748,0.636399,0.263651,0.636399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-05,34.792770,34.483752,34.647398,NaN,NaN,0.309018,0.127370,0.436388,0.436388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-06,34.765586,34.447387,34.683764,NaN,NaN,0.318199,0.118188,0.200011,0.318199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-07,34.874683,34.265558,34.456478,NaN,NaN,0.609125,0.190919,0.418206,0.609125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-10,34.556484,34.292832,34.383747,NaN,NaN,0.263652,0.100006,0.163646,0.263652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-11,34.756495,34.292832,34.565575,NaN,NaN,0.463663,0.372748,0.090915,0.463663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
